# Sora2 Watermark Remover (Google Colab版)

このノートブックは、Sora動画からウォーターマークを除去するツールのGoogle Colab版です。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. セットアップ

### GitHubからリポジトリをクローン

In [ ]:
# GitHubリポジトリのクローン
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover

# ロールバック版ブランチ（GitHub30基準の安定版）に切り替え
!git checkout revert/github30

print("\n✅ リポジトリのクローンが完了しました。")

### ⚠️ 重要: Pillow依存関係の修正

**🔴 重要: 以下のセルを実行後、必ずランタイムを再起動してください！**

1. **下のセルを実行**
2. **「ランタイム」→「ランタイムを再起動」を選択（必須！）**
3. **再起動後、次のセルから実行を続ける**

⚠️ **再起動しないとpathlibエラーが発生します**

In [ ]:
# Pillow依存関係の修正（実行後にランタイムを再起動してください）
!pip uninstall -y pillow PIL --no-cache-dir --quiet
!pip install --no-cache-dir --quiet "pillow==10.4.0"
!pip install --no-cache-dir --quiet "websockets==15.0.1"

print("\n✅ 完了しました。")
print("\n🔴 重要: ランタイム→ランタイムを再起動 を今すぐ実行してください！")
print("再起動しないとpathlibエラーが発生します。")

### ⬇️ ランタイム再起動後、ここから続行

In [ ]:
# Path動作確認（再起動後の最初のセル）
from pathlib import Path
print("Path動作テスト:", Path("/content").exists())  # True ならOK

if Path("/content").exists():
    print("✅ Pathlib正常動作を確認")
else:
    print("❌ エラー: ランタイムを再起動してください")

In [ ]:
# ディレクトリ移動（再起動後は必要）
%cd /content/Sora2WatermarkRemover
print("✅ 作業ディレクトリに移動しました")

In [ ]:
# Pillow確認
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow依存関係は正常です。次のセルに進んでください。")
else:
    print("⚠️ 問題があります。ランタイムを再起動しましたか？")

In [ ]:
# 必要なパッケージのインストール
!pip install --no-cache-dir --quiet transformers torch opencv-python tqdm loguru iopaint
!apt-get -qq install -y ffmpeg

# LaMa モデルの事前ダウンロード（重要！）
print("LaMaモデルをダウンロード中...")
!iopaint download --model lama

print("\n✅ パッケージ＋LaMaモデルの準備が完了しました。")

## 2. デバイス確認

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPUが利用できません。ランタイム→ランタイムのタイプを変更→GPUを選択してください。")

## 3. 動画アップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

# アップロードされたファイルを取得
input_video = list(uploaded.keys())[0]
print(f"\nUploaded: {input_video}")

## 4. 動画処理

### パラメータ設定

- `max_bbox_percent`: バウンディングボックスの最大サイズ（画像に対する割合%）
- `frame_step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `target_fps`: 出力fps（0=入力と同じ）

In [ ]:
# パラメータ設定（必要に応じて調整）
max_bbox_percent = 10.0  # バウンディングボックスの最大サイズ
frame_step = 1           # 1=全フレーム、2=隔フレーム
target_fps = 0.0         # 0=入力と同じ

output_video = "output_no_watermark.mp4"

# ディレクトリ移動
%cd /content/Sora2WatermarkRemover

# 入力ファイルの確認
import os
input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

if not os.path.exists(input_path):
    print(f"❌ エラー: 入力ファイルが見つかりません: {input_path}")
else:
    print(f"✅ 入力ファイル確認: {input_path}")
    print(f"📝 出力先: {output_path}")
    print(f"⚙️ パラメータ: max_bbox_percent={max_bbox_percent}, frame_step={frame_step}, target_fps={target_fps}")
    print("\n🚀 処理を開始します...\n")
    
    # remwm.pyを実行
    !python remwm.py "{input_path}" "{output_path}" \
      --max-bbox-percent {max_bbox_percent} \
      --frame-step {frame_step} \
      --target-fps {target_fps}
    
    # 出力ファイルの確認
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"\n✅ 処理完了: {output_path}")
        print(f"📊 ファイルサイズ: {file_size:.2f} MB")
    else:
        print(f"\n❌ エラー: 出力ファイルが作成されませんでした")
        print("上記のエラーメッセージを確認してください")

## 5. 結果のダウンロード

In [ ]:
# 結果のダウンロード
print("結果をダウンロード:")
files.download(f"/content/{output_video}")

## 6. 使い方のヒント

### 処理速度を上げたい場合

```python
frame_step = 2  # 隔フレーム処理（約2倍速）
```

### 出力fpsを固定したい場合

```python
target_fps = 30  # 30fpsで出力
```

### 組み合わせ例

```python
frame_step = 2
target_fps = 30
# → 隔フレーム処理して30fpsで出力
```

## 7. トラブルシューティング

### ⚠️ pathlibエラー（AttributeError: 'PosixPath' object has no attribute '_str'）

**解決方法:**
1. **ランタイムを再起動してください（必須）**
   - Pillow再インストール直後の状態で起きる典型症状です
2. **再起動後は次の順で実行:**
   1) リポジトリ/ブランチ切替セル（revert/github30）
   2) Pillow確認セル（version=10.4.0, is_directory=True を確認）
   3) 依存＋LaMaダウンロードセル
   4) 以降の処理セル

### その他のよくある問題

1. **pipインストールがハングアップする場合**
   - ランタイムを再起動してもう一度実行

2. **LaMaモデルが読み込めない場合**
   - `!iopaint download --model lama`を再実行
   - ランタイムを再起動

3. **GPUが使用できない場合**
   - ランタイム→ランタイムのタイプを変更→GPU（T4推奨）を選択

4. **メモリエラーの場合**
   - `frame_step`を大きくして処理フレーム数を減らす
   - 動画を短く分割して処理

### デバッグ用テストコード

```python
# Path動作確認
from pathlib import Path
print(Path("/content").exists())  # True ならOK
```